In [ ]:
#importo librerías

import pandas as pd
from fastapi import FastAPI
import ast


In [ ]:
#cardo los datasets

credits = pd.read_csv("Dataset/credits.csv")

movies_dataset = pd.read_excel("Dataset/movies_dataset.xlsx")

In [ ]:
# TRANSFORMACIONES - Primer Transformación, desanidar columnas


def convert(obj):
    if isinstance(obj, str) and '{' in obj: 
        L=[]
        for i in ast.literal_eval(obj): 
            L.append(i['name']); 
        return L



def convert2(obj):
    if isinstance(obj, str) and '{' in obj:
        dic = ast.literal_eval(obj)
        return dic['name']



In [ ]:
movies_dataset['genres'] = movies_dataset['genres'].apply(convert)
movies_dataset['belongs_to_collection'] = movies_dataset['belongs_to_collection'].apply(convert2)
movies_dataset['production_companies'] = movies_dataset['production_companies'].apply(convert)
movies_dataset['production_countries'] = movies_dataset['production_countries'].apply(convert)
movies_dataset['spoken_languages'] = movies_dataset['spoken_languages'].apply(convert)

In [ ]:
# Siguientes transformaciones, completar los faltantes son 0 en las columnas revenue y budget

movies_dataset["revenue"] = movies_dataset["revenue"].fillna(0)
movies_dataset["budget"] = movies_dataset["budget"].fillna(0)


In [ ]:
# Eliminar valores nulos en la columna "release_date"
movies_dataset.dropna(subset=["release_date"], inplace=True)

# Convertir la columna de fecha al formato de fecha de Pandas
movies_dataset['release_date'] = pd.to_datetime(movies_dataset['release_date'], errors='coerce')

# Crear la columna "release_year" con el año de la fecha
movies_dataset['release_year'] = movies_dataset['release_date'].dt.year



In [ ]:
#Crear la columna con el retorno de inversión, llamada return con los campos revenue y budget, dividiendo estas dos últimas revenue / budget,
#cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0.

# Definir la función para calcular el retorno de inversión
def calculate_return(row):
    
    if row['budget'] == 0:
        return 0
    
    elif row["revenue"] == 0:
        return 0
    
    else:
        return row['revenue'] / row['budget']
    


movies_dataset['return'] = movies_dataset.apply(calculate_return, axis=1)



In [ ]:

# Elimino columnas no utilizadas
movies_dataset = movies_dataset.drop(columns = ["video","imdb_id","adult","original_title","poster_path","homepage"])

In [ ]:
# hago un duplicado del df para trabajar mejor
credits2 = credits

credits2 = credits2.drop(columns=["cast"])

credits2["crew"] = credits2["crew"].apply(ast.literal_eval)

In [ ]:
# extraigo los nombres de los directores de cada pelicula  y los pongo en una lista. Luego cambio los valores de la columna crew por esas listas

def obtener_nombres_directores(lista):
    nombres_directores = []
    if isinstance(lista, list):
        for diccionario in lista:
            if isinstance(diccionario, dict) and diccionario.get("job") == "Director":
                nombre_director = diccionario.get("name")
                if nombre_director:
                    nombres_directores.append(nombre_director)
    return nombres_directores

In [ ]:
credits2["crew"] = credits2["crew"].apply(obtener_nombres_directores)

In [ ]:
# hacemos un merge para combinar las dataframe

movies_dataset_merged = movies_dataset.merge(credits2, on="id")

In [ ]:
movies_dataset_merged.head()

In [ ]:
movies_dataset_merged.to_csv('Dataset/df.csv', index=False)
